In [1]:
import requests
import ast
import pandas as pd
import numpy as np
import sys
sys.path.append("../")
from scr.driver import driver
from scr.infoproducts_pull import get_id_info
from scr.stores import get_stores_pull
from time import sleep

In [4]:
product= 4470305300

store = {
    "physicalStoreId":13636
}


stock = "https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/{}"
headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
res = requests.get(stock.format(f"0{product}"), params=store, headers=headers).json()

In [5]:
res

{'stocks': [{'datatype': 'stock',
   'physicalStoreId': 13636,
   'sizeStocks': [{'datatype': 'sizeStock',
     'sizeId': 2,
     'quantity': 0,
     'size': 102},
    {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 1, 'size': 103},
    {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}]}

In [6]:
def get_stock_pull(search_terms,lat,lng):
        

        products = get_id_info(search_terms)

        

        store_info = get_stores_pull(lat, lng)
        store_ids = [element["id"] for element in store_info]

        stock_url = "https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/{}"
        headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
        stock_params = {"physicalStoreId": store_ids}

        urls = [stock_url.format("".join(p["id_prod"]).rjust(11,"0")) for p in products]
        print(urls)
        
        for i,product in enumerate(products):
            headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
            products[i]["stock"] = requests.get(urls[i],params=stock_params,headers=headers).json()
        
        return products

In [18]:
prod = get_stock_pull("chaqueta amarilla",40.416050951604056, -3.7072654155715603)

Cazadora confeccionada en 100% algodón con detalle de bolsillos delanteros de tapeta, cuello clásico y manga larga.
Chaleco de ochos corto con cuello de pico y detalle de rib en el bajo.
['https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/05713324309', 'https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/04574303300']


In [19]:
prod

[{'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502620977.html?colorId=309',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/5713/324/309/5713324309_2_1_2.jpg?t=1619715493366',
  'product_name': 'Cazadora algodón bolsillos delanteros',
  'price': '25,99 € ',
  'id_prod': ['5713324309'],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 13636,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 4,
       'quantity': 1,
       'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5239,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 1,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 1, 'size': 104}]}]}},
 {'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502660734.html?colorId=300',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/4574/303/300/4574303300_2_1_2.jpg?t=1613464621559',
  'product_name': 'Chaleco ochos corto',
  'pric

In [7]:
def sacar_cosas(diccionario):
    
    url  = diccionario.get('url')
    image  = diccionario.get('image')
    product_name  = diccionario.get('product_name')
    price  = diccionario.get('price')
    
    items = {"Product_name": [], "url": [], "Image":[], "Price":[], "Quantity":[], "Size":[], "Tienda": []}
    
    size_ = {101:"XS", 102:"S", 103:"M", 104:"L", 105:"XL", 106:"XXL",125:"S-M",131:"L-XL"}
    items["url"].append(url)
    
    for k,v in diccionario["stock"].items():
        print((v[0]))
        tienda = v[0].get('physicalStoreId')
    for i in v[0]["sizeStocks"]:
        #size = i["sizeStocks"].get('sizeId')
        print(i)
        size  = i.get('size')
        converted_size = size_[size]
        quantity  = i.get('quantity')
        
        if quantity <= 0:
            quantity = "Lo siento, no tenemos existencias!☹️"
        elif quantity == 1:
            quantity = "Últimas existencias 👀"
                
        elif quantity >= 2:
            quantity = "El producto está en tienda!🙂"
                
        elif quantity >= 4:
            quantity = "Suficientes existencias!!😁"
        
        
        items["Quantity"].append(quantity)
        items["Size"].append(converted_size)
        items["url"].append(url)
        items["Image"].append(image)
        items["Product_name"].append(product_name)
        items["Price"].append(price)
        items["Tienda"].append(tienda)
        
        
                
    return items

In [10]:
def productos_finales(search_items, lat, lon):
    prod = get_stock_pull(search_items,lat,lon)
    i_busqueda = list(map(sacar_cosas, prod))
    df = pd.DataFrame(i_busqueda)
    df2 = df.explode("Size")
    df3 = df2.explode("Quantity")
    df4 = df3.drop_duplicates(subset=["Quantity", "Size"])
    return df4 

In [9]:
def limpieza_df():
    df2 = df.explode("Size")
    df3 = df2.explode("Quantity")
    df4 = df3.drop_duplicates(subset=["Quantity", "Size"])
    return df4